# Meta-Learning 测试 Notebook (Colab版本)

本notebook专门用于测试Meta-Learning方法，训练RNN成为"快速学习器"，
使其能够从1D规则快速适应并泛化到2D组合任务。

## 🚀 Colab使用说明

1. **启用GPU**: Runtime -> Change runtime type -> GPU (T4/V100/A100)
2. **上传项目**: 将项目文件上传到Colab，或使用GitHub克隆
3. **重要**: 如果遇到NumPy版本错误，请先运行Cell 3修复NumPy兼容性
4. **运行所有Cells**: 按顺序运行所有代码块

## ⚠️ 常见问题

**如果遇到 `ValueError: numpy.dtype size changed` 错误**：
1. 先运行Cell 3（修复NumPy兼容性）
2. 重启运行时：Runtime -> Restart runtime
3. 然后重新运行Cell 3和后续cells

## 核心思想

1. **Meta-Training (外循环)**: 训练RNN学习"如何学习"
   - 通过大量随机任务，让RNN学习快速适应新任务
   - RNN通过隐藏状态适应新任务，而不改变权重θ

2. **Meta-Testing (内循环)**: 在真实任务上测试
   - 冻结权重θ_final
   - 通过1D支持集适应（模拟人类学习）
   - 在2D查询集上测试（模拟人类测试）


### 📦 在Colab中准备项目文件

**如果这是第一次在Colab中运行，请先执行以下步骤之一：**

**选项1: 从GitHub克隆（如果项目在GitHub上）**
```python
# 取消下面的注释并运行
# !git clone https://github.com/your-username/wine-spave.git
# %cd wine-spave
```

**选项2: 上传项目文件**
- 将项目文件夹压缩为zip文件
- 运行下面的cell上传并解压


In [ ]:
# 如果需要在Colab中上传项目文件，取消下面的注释并运行
# from google.colab import files
# import zipfile
# import os
# 
# # 上传zip文件
# uploaded = files.upload()
# 
# # 解压文件
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content')
#         print(f"✓ 已解压 {filename}")
#         # 切换到项目目录
#         project_name = filename.replace('.zip', '')
#         if os.path.exists(f'/content/{project_name}'):
#             os.chdir(f'/content/{project_name}')
#             print(f"✓ 已切换到项目目录: {os.getcwd()}")
# 
# print("如果已上传项目，请继续运行下一个cell")


In [ ]:
# 修复Colab中的NumPy兼容性问题
# 这个cell必须在导入torchvision之前运行
import sys
import subprocess

# 检查是否在Colab环境
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 修复NumPy版本兼容性...")
    try:
        import numpy as np
        numpy_version = np.__version__
        print(f"当前NumPy版本: {numpy_version}")
        
        # 如果是NumPy 2.x，需要降级到1.x以兼容PyTorch
        if numpy_version.startswith('2.'):
            print("检测到NumPy 2.x，降级到NumPy 1.26.4以兼容PyTorch...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--force-reinstall", "numpy==1.26.4"])
            # 清除numpy模块缓存
            if 'numpy' in sys.modules:
                del sys.modules['numpy']
            import numpy as np
            print(f"✓ NumPy已降级到版本: {np.__version__}")
        else:
            print(f"✓ NumPy版本 {numpy_version} 应该兼容")
    except Exception as e:
        print(f"检查NumPy时出错: {e}")
        print("安装NumPy 1.26.4...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "--force-reinstall", "numpy==1.26.4"])
        import numpy as np
        print(f"✓ NumPy安装完成: {np.__version__}")
    
    print("✓ NumPy兼容性检查完成\n")


## 1. 环境设置和导入

**注意**: 如果这是第一次运行，请先运行此cell安装依赖并设置环境。


In [ ]:
# Colab环境设置
import os
import sys

# 检查是否在Colab环境
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🚀 检测到Colab环境")
    # 安装必要的库（在Colab中使用!pip是标准做法）
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numpy<2", "torch", "torchvision", "matplotlib"])
    
    # 克隆或上传项目（根据实际情况选择）
    # 方法1: 如果项目在GitHub上，可以克隆
    # import subprocess
    # subprocess.check_call(['git', 'clone', 'https://github.com/your-repo/wine-spave.git'])
    # os.chdir('wine-spave')
    
    # 方法2: 如果项目需要上传，使用以下代码
    # from google.colab import files
    # uploaded = files.upload()  # 上传项目zip文件
    # import zipfile
    # with zipfile.ZipFile('wine-spave.zip', 'r') as zip_ref:
    #     zip_ref.extractall('/content')
    # os.chdir('/content/wine-spave')
    
    # 方法3: 如果项目已经在Colab中，直接使用当前目录
    project_root = '/content'  # 或你的项目路径
    if os.path.exists('/content') and 'wine-spave' in os.listdir('/content'):
        project_root = '/content/wine-spave'
        os.chdir(project_root)
else:
    # 本地环境
    import subprocess
    current_dir = os.getcwd()
    if 'notebooks' in current_dir:
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir
    
    # 检查并安装必要的库
    try:
        import numpy as np
        print("✓ NumPy已安装")
    except ImportError:
        print("NumPy未安装，正在安装...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numpy<2"])
        import numpy as np
        print("✓ NumPy安装完成")
    
    try:
        import torch
        print("✓ PyTorch已安装")
    except ImportError:
        print("PyTorch未安装，正在安装...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision"])
        import torch
        print("✓ PyTorch安装完成")
    
    try:
        import matplotlib.pyplot as plt
        print("✓ Matplotlib已安装")
    except ImportError:
        print("Matplotlib未安装，正在安装...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "matplotlib"])
        import matplotlib.pyplot as plt
        print("✓ Matplotlib安装完成")

import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 添加项目根目录到路径
if not IN_COLAB:
    if project_root not in sys.path:
        sys.path.insert(0, project_root)
else:
    if project_root not in sys.path:
        sys.path.insert(0, project_root)

print(f"\n项目根目录: {project_root}")

# GPU检测和设置
if torch.cuda.is_available():
    device = "cuda"
    print(f"✓ GPU可用: {torch.cuda.get_device_name(0)}")
    print(f"  GPU数量: {torch.cuda.device_count()}")
    print(f"  当前GPU: {torch.cuda.current_device()}")
    print(f"  显存: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    device = "cpu"
    print("⚠ GPU不可用，将使用CPU（速度较慢）")
    if IN_COLAB:
        print("  提示: 在Colab中，请确保已启用GPU: Runtime -> Change runtime type -> GPU")

# 导入项目模块
print("\n导入项目模块...")
from data import GridDataGenerator
from models import get_model
from country_task import get_country_task_loaders
from meta_learning import (
    meta_train, 
    meta_test, 
    create_meta_learning_args,
    SequentialRNN
)

print("✓ 所有模块导入完成")


✓ NumPy已安装
✓ PyTorch已安装
Matplotlib未安装，正在安装...



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip


✓ Matplotlib安装完成

项目根目录: /Users/yifei/Desktop/UCLA/wine-spave

导入项目模块...
✓ 所有模块导入完成


## 2. 参数设置


In [ ]:
# 基础参数类
class Args:
    def __init__(self):
        # 设备设置（自动检测GPU）
        self.use_cuda = torch.cuda.is_available()
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.seed = 42
        
        # 数据集设置
        self.use_images = False  # 使用索引而不是图像
        self.image_dir = 'images/faces16'
        self.training_regime = 'ungrouped'
        self.grid_size = 4
        self.ctx_order = 'first'
        self.inner_4x4 = False
        
        # 训练设置
        self.bs = 16  # Batch size（用于country task等）
        
        # 模型设置
        self.model_name = 'rnn'  # 使用RNN/LSTM
        self.ctx_scale = 1.0
        self.measure_grad_norm = False
        
        # Meta-Learning参数
        self.meta_lr = 0.001  # Meta-learning学习率
        self.n_support = 16   # 每个任务的1D支持样本数
        self.n_query = 32     # 每个任务的2D查询样本数
        self.n_meta_iterations = 1000  # Meta-training迭代次数（测试用）
        self.n_tasks_per_batch = 4    # 每批任务数

# 创建参数对象
args = Args()

# 设置随机种子
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

print("参数设置完成！")
print(f"设备: {args.device} ({'GPU加速' if args.use_cuda else 'CPU'})")
print(f"模型: {args.model_name}")
print(f"Meta学习率: {args.meta_lr}")
print(f"Support集大小: {args.n_support}")
print(f"Query集大小: {args.n_query}")
print(f"Meta迭代次数: {args.n_meta_iterations}")
print(f"每批任务数: {args.n_tasks_per_batch}")

if args.use_cuda:
    print(f"\n🚀 将使用GPU加速训练（预计速度提升10-100倍）")
else:
    print(f"\n⚠ 使用CPU训练（速度较慢，建议在Colab中启用GPU）")


参数设置完成！
模型: rnn
Meta学习率: 0.001
Support集大小: 16
Query集大小: 32
Meta迭代次数: 1000
每批任务数: 4


## 3. 准备测试数据（Country Task）


In [8]:
# 生成grid数据（用于meta-testing的真实任务）
print("生成测试数据...")
grid = GridDataGenerator(
    training_regime=args.training_regime,
    size=args.grid_size,
    use_images=args.use_images,
    image_dir=args.image_dir,
    inner_4x4=args.inner_4x4
)

print(f"训练样本数: {len(grid.train)}")
print(f"测试样本数: {len(grid.test)}")

# 创建country task数据
country_loader, country_data_gen = get_country_task_loaders(args, grid)

print(f"\n✓ Country Task数据创建完成")
print(f"  国家数量: {country_data_gen.n_countries}")
print(f"  总样本数: {len(country_loader.dataset)}")

# 显示国家列表
print(f"\n国家列表:")
print("1D国家 (4个):")
for i, country_info in enumerate(country_data_gen.countries):
    if country_info[1] == 'single':
        name = country_info[0]
        attrs = country_info[2]
        direction = country_info[3]
        dir_desc = "高更好" if direction == 1 else "低更好"
        print(f"  {i}. {name}: 上下文{attrs[0]} ({dir_desc})")

print("\n2D国家 (4个):")
for i, country_info in enumerate(country_data_gen.countries):
    if country_info[1] == 'dual':
        name = country_info[0]
        attrs = country_info[2]
        direction = country_info[3]
        dir1_desc = "高" if direction[0] == 1 else "低"
        dir2_desc = "高" if direction[1] == 1 else "低"
        print(f"  {i}. {name}: 上下文{attrs[0]}({dir1_desc})+上下文{attrs[1]}({dir2_desc})")


生成测试数据...
训练样本数: 192
测试样本数: 192

✓ Country Task数据创建完成
  国家数量: 8
  总样本数: 256

国家列表:
1D国家 (4个):
  0. France: 上下文0 (高更好)
  1. Italy: 上下文0 (低更好)
  2. China: 上下文1 (高更好)
  3. Australia: 上下文1 (低更好)

2D国家 (4个):
  4. Russia: 上下文0(高)+上下文1(高)
  5. Brazil: 上下文0(高)+上下文1(低)
  6. India: 上下文0(低)+上下文1(高)
  7. Mexico: 上下文0(低)+上下文1(低)


In [ ]:
# 如果想临时修改迭代次数，可以在这里直接修改
# 例如：只训练500次用于快速测试
args.n_meta_iterations = 500
print(f"✓ 已设置迭代次数为: {args.n_meta_iterations}")


## 4. Meta-Training (外循环)

训练RNN成为一个"快速学习器"。这个过程会：
- 生成大量随机任务（每个任务是一个新的4x4地图）
- 每个任务包含1D支持集和2D查询集
- RNN通过隐藏状态适应每个任务，然后基于查询集的表现更新权重


In [ ]:
# 创建meta-learning参数
meta_args = create_meta_learning_args(args)

# 创建新模型用于meta-training
print("创建模型...")
meta_model = get_model(meta_args)
meta_model.to(meta_args.device)

print(f"模型参数数量: {sum(p.numel() for p in meta_model.parameters()):,}")
print(f"设备: {meta_args.device}")
print(f"\n开始Meta-Training...")
print(f"  迭代次数: {meta_args.n_meta_iterations}")
print(f"  每批任务数: {meta_args.n_tasks_per_batch}")
print(f"  Support集大小: {meta_args.n_support}")
print(f"  Query集大小: {meta_args.n_query}")

if meta_args.use_cuda:
    print(f"\n🚀 使用GPU加速，预计训练时间: {meta_args.n_meta_iterations * 0.1:.1f}秒 - {meta_args.n_meta_iterations * 1:.1f}秒")
else:
    print(f"\n⚠ 使用CPU，预计训练时间: {meta_args.n_meta_iterations * 5:.1f}秒 - {meta_args.n_meta_iterations * 30:.1f}秒")
    print(f"   建议在Colab中启用GPU以加速训练")

print(f"\n注意: 这个过程可能需要较长时间")
print(f"      实际使用时，建议设置 n_meta_iterations >= 10000\n")


创建模型...
Model is an LSTM
模型参数数量: 83778

开始Meta-Training...
  迭代次数: 10000
  每批任务数: 4
  Support集大小: 16
  Query集大小: 32

注意: 这个过程可能需要较长时间
      实际使用时，建议设置 n_meta_iterations >= 10000



In [10]:
# 执行meta-training
meta_trained_model, meta_losses = meta_train(
    meta_model, 
    meta_args,
    n_meta_iterations=meta_args.n_meta_iterations,
    n_tasks_per_batch=meta_args.n_tasks_per_batch
)


开始Meta-Training...
  总迭代次数: 10000
  每批任务数: 4
  Support set大小: 16
  Query set大小: 32



KeyboardInterrupt: 

### 4.1 可视化Meta-Training损失


In [ ]:
# 可视化meta-training损失
plt.figure(figsize=(12, 5))

# 原始损失
plt.subplot(1, 2, 1)
plt.plot(meta_losses, alpha=0.6, linewidth=1)
plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Raw)')
plt.grid(True, alpha=0.3)

# 移动平均
plt.subplot(1, 2, 2)
window = min(100, len(meta_losses) // 10)
if window > 1 and len(meta_losses) >= window:
    moving_avg = np.convolve(meta_losses, np.ones(window)/window, mode='valid')
    plt.plot(meta_losses, alpha=0.3, label='Raw', linewidth=1)
    plt.plot(range(window-1, len(meta_losses)), moving_avg, 
             label=f'Moving Average (window={window})', linewidth=2)
    plt.xlabel('Meta-Iteration')
plt.ylabel('Meta-Loss')
plt.title('Meta-Training Loss (Smoothed)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"初始Meta-Loss: {meta_losses[0]:.4f}")
print(f"最终Meta-Loss: {meta_losses[-1]:.4f}")
if len(meta_losses) > 100:
    print(f"最后100次平均: {np.mean(meta_losses[-100:]):.4f}")


## 5. Meta-Testing (内循环)

在真实的国家任务上测试meta-trained RNN：
1. 冻结模型权重
2. 对每个国家：
   - 使用1D支持集进行适应（模拟人类学习阶段）
   - 在2D查询集上测试（模拟人类测试阶段）
3. 比较RNN表现与人类基线


In [ ]:
print("开始Meta-Testing...")
print("  模型权重已冻结")
print("  测试真实国家任务\n")

# 执行meta-testing
meta_test_results = meta_test(
    meta_trained_model,
    meta_args,
    grid,
    country_data_gen,
    country_loader
)


## 6. 结果分析


In [ ]:
print("\n" + "="*60)
print("📊 Meta-Testing 结果总结")
print("="*60)

# 按国家显示结果
print("\n按国家分类:")
for country_name, acc in meta_test_results.items():
    print(f"  {country_name:12s}: {acc:.4f} ({acc*100:6.2f}%)")

# 计算平均准确率
avg_acc = sum(meta_test_results.values()) / len(meta_test_results)
print(f"\n平均准确率: {avg_acc:.4f} ({avg_acc*100:.2f}%)")

# 按偏好类型分类
single_countries = [name for i, name in enumerate(country_data_gen.countries) 
                   if country_data_gen.countries[i][1] == 'single']
dual_countries = [name for i, name in enumerate(country_data_gen.countries) 
                 if country_data_gen.countries[i][1] == 'dual']

single_accs = [meta_test_results[name] for name in single_countries if name in meta_test_results]
dual_accs = [meta_test_results[name] for name in dual_countries if name in meta_test_results]

if single_accs:
    single_avg = sum(single_accs) / len(single_accs)
    print(f"\n1D国家平均准确率: {single_avg:.4f} ({single_avg*100:.2f}%)")
if dual_accs:
    dual_avg = sum(dual_accs) / len(dual_accs)
    print(f"2D国家平均准确率: {dual_avg:.4f} ({dual_avg*100:.2f}%)")

# 基线比较
print(f"\n基线比较:")
print(f"  随机猜测基线: 33.33% (3选1)")
print(f"  人类基线: ~66-80% (根据论文)")

# 评估结果
print(f"\n评估:")
if avg_acc > 0.66:
    print(f"  ✓ Meta-Learning成功！准确率({avg_acc*100:.2f}%)超过人类基线下限")
elif avg_acc > 0.33:
    print(f"  ⚠ Meta-Learning部分成功，准确率({avg_acc*100:.2f}%)超过随机但未达到人类基线")
else:
    print(f"  ✗ Meta-Learning需要进一步优化，准确率({avg_acc*100:.2f}%)接近随机水平")


### 6.1 可视化结果


In [ ]:
# 可视化meta-testing结果
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 1. 各国家准确率
ax1 = axes[0]
country_names = list(meta_test_results.keys())
country_accs = list(meta_test_results.values())

# 按类型着色
colors = []
for name in country_names:
    country_idx = [i for i, c in enumerate(country_data_gen.countries) if c[0] == name][0]
    if country_data_gen.countries[country_idx][1] == 'single':
        colors.append('#3498db')  # 蓝色：1D国家
    else:
        colors.append('#e74c3c')  # 红色：2D国家

bars = ax1.bar(range(len(country_names)), country_accs, color=colors, alpha=0.7, 
                edgecolor='black', linewidth=1.5)

for i, (bar, acc) in enumerate(zip(bars, country_accs)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.2f}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

ax1.set_xticks(range(len(country_names)))
ax1.set_xticklabels(country_names, rotation=45, ha='right')
ax1.set_ylabel('准确率', fontweight='bold')
ax1.set_title('Meta-Testing: 各国家准确率', fontweight='bold', fontsize=12)
ax1.set_ylim([0, 1.1])
ax1.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5, label='随机猜测 (33%)')
ax1.axhline(y=0.66, color='green', linestyle='--', alpha=0.5, label='人类基线下限 (66%)')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# 2. 按类型分类
ax2 = axes[1]
categories = ['总体', '1D国家', '2D国家']
accs = [avg_acc]
if single_accs:
    accs.append(single_avg)
else:
    accs.append(0)
if dual_accs:
    accs.append(dual_avg)
else:
    accs.append(0)

colors_cat = ['#2ecc71', '#3498db', '#e74c3c']
bars2 = ax2.bar(categories, accs, color=colors_cat, alpha=0.7, 
                edgecolor='black', linewidth=1.5)

for bar, acc in zip(bars2, accs):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{acc:.3f}\n({acc*100:.1f}%)',
            ha='center', va='bottom', fontweight='bold')

ax2.set_ylabel('准确率', fontweight='bold')
ax2.set_title('Meta-Testing: 按类型分类', fontweight='bold', fontsize=12)
ax2.set_ylim([0, 1.1])
ax2.axhline(y=0.33, color='gray', linestyle='--', alpha=0.5, label='随机猜测')
ax2.axhline(y=0.66, color='green', linestyle='--', alpha=0.5, label='人类基线')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✓ 可视化完成")


## 7. 总结

### Meta-Learning成功标准

如果meta-learning成功，RNN应该能够：
- ✅ 在**从未见过任何2D规则训练**的情况下（零样本）
- ✅ 在2D国家任务上表现出**远高于随机水平**的准确率
- ✅ 成功模拟人类的组合泛化能力

### 性能基线
- **随机猜测**: 33.33% (3选1)
- **人类基线**: ~66-80% (根据论文)
- **目标**: >66% (超过人类基线下限)

### 下一步
1. 如果准确率低于预期，可以尝试：
   - 增加meta-training迭代次数（≥10000）
   - 调整meta-learning rate
   - 优化support set和query set的大小
   - 改进任务生成策略

2. 如果准确率满足要求，可以：
   - 进行更深入的分析
   - 与人类数据对比
   - 探索不同超参数的影响
